# 1. SETTINGS

In [56]:
# libraries
import numpy as np
import pandas as pd
from datetime import date

In [57]:
# warnings
import warnings
warnings.filterwarnings("ignore")

In [58]:
# pandas options
pd.set_option("display.max_columns", None)

# 2. IMPORT

In [64]:
# import datasets
data = pd.read_csv("../data/prepared/data_basic.csv")

In [65]:
# check data
print("Dimensions:", data.shape)
data.head()

Dimensions: (24497508, 6)


,PredictionIdx,CustomerIdx,IsinIdx,BuySell,CustomerInterest,Week
0,NaN,0,16471,Buy,0.0,71
1,NaN,0,16471,Buy,1.0,72
2,NaN,0,16471,Buy,0.0,73
3,NaN,0,16471,Buy,0.0,74
4,NaN,0,16471,Buy,0.0,75


# 3. CREATE FEATURES

### RECENCY (TIME SINCE LAST INTERACTION)

In [10]:
# function to compute recency
def compute_recency(data, group_id):

    # sorting
    tmp = data[data.CustomerInterest == 1]
    tmp.sort_values(by = ["Week"], ascending = True)

    # group and subset
    tmp["PrevWeek"] = tmp.groupby(group_id).Week.shift(1)

    # merge to data
    data = data.merge(tmp, how = "left")

    # fill NA
    tmp = data.groupby(group_id).PrevWeek.fillna(method = 'ffill', inplace = True)
    tmp.PrevWeek.fillna("ffil", inplace = True)
    data["PrevWeek"] = tmp["PrevWeek"]

    # compute recency
    data["Recency"] = data["Week"] - data["PrevWeek"]
    del data["PrevWeek"]
    return(data)

In [11]:
# computy recency
data = compute_recency(data, group_id = ["CustomerIdx", "IsinIdx", "BuySell"])
data = compute_recency(data, group_id = ["CustomerIdx", "IsinIdx"])
data = compute_recency(data, group_id = ["CustomerIdx"])
data = compute_recency(data, group_id = ["IsinIdx"])

KeyError: 'Column not found: False'

In [ ]:
data[(data.CustomerIdx == 11) & (data.IsinIdx == 23218)].head(7)

### FREQUENCY (TOTAL NUMBER OF INTERACTIONS)

### MONTH AND WEEK ID

# 4. EXPORT

In [ ]:
# check dimensions
print(data.shape)

In [ ]:
# check NA
data.isnull().sum()

In [ ]:
# export CSV
data.to_csv("../data/prepared/data_v1.csv", index = False)